In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import mdtraj as md
import numpy as np
import seaborn as sns

from pprint import pprint

from metamds.db import retrieve_all, query_sim
from metamds.db import update_doc
# from block_avg import block_average

In [2]:
collection = "peg_oxy_cap"
#params = dict()
#params['polymer_class'] = ["AlkaneMonolayer"]
#params['surface_class'] = ["Betacristobalite"]
#params['chain_length'] = [10]
#params['normal_force(kJ/mol/A)'] = [1500]
#cursor = query_sim(**params, collection=collection)
cursor = retrieve_all(collection=collection)

In [3]:
for doc in cursor:
    if 'stick_slip' in doc.keys():
        continue
    trr_file = '{}shear_whole.xtc'.format(doc['output_dir'])
    top_file = '{}nvt.gro'.format(doc['output_dir'])
    
    polymer = doc['polymer_class']
    n_molecules = doc ['n_molecules']
    chain_length = doc['chain_length']
    surface = doc['surface_class']
    pattern = doc['pattern_class']
    normal_force = doc['normal_force(kJ/mol/A)']

    name = doc['name']
    
    traj = md.load(trr_file, top=top_file)
    
    dt = traj.time[1] - traj.time[0]
    toss = 1000/dt # 1 ns

    t = traj.time[toss:]
    x = np.copy(traj.xyz[toss:, traj.n_atoms/2 + 2, 0])
    plt.plot(t, x)
    plt.xlabel('time (ps)')
    plt.ylabel('x position (nm)')
    print(name, normal_force)
    plt.show()
    plt.clf()
    
    box_x = traj.unitcell_lengths[0, 0]

    n_hops = np.zeros_like(x)
    for i, p in enumerate(x):
        if p - x[i-1] > 0.5*box_x:
            n_hops[i:] += 1
        elif p - x[i-1] < -0.5*box_x:
            n_hops[i:] -= 1

    x_no_hops = x - n_hops * box_x
    
    x_no_hops = x_no_hops.tolist()
    t = t.tolist()
    
    stick_slip = {"stick_slip":[x_no_hops, t]}
    update_doc(existing_doc=doc, added_values=stick_slip, collection=collection)
    
    fig = plt.figure()
    #plt.plot(t[:20000] / 1000, x_no_hops[:20000])
    plt.plot([x / 1000 for x in t[2400:2500]], x_no_hops[2400:2500])
    plt.xlabel('time (ns)')
    plt.ylabel('x position (nm)')
    plt.show()
    plt.clf()
    del traj
#fig.savefig('x_displacement.pdf', bbox_inches='tight')